In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from functools import partial
%matplotlib inline

In [2]:
df_full = pd.read_csv('../data/LoanStats2014.csv', skiprows=1, skipfooter=2)

C:\Users\Daniel\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  if __name__ == '__main__':


In [3]:
df = df_full.copy()

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 235629 entries, 0 to 235628
Columns: 111 entries, id to total_il_high_credit_limit
dtypes: float64(44), int64(43), object(24)
memory usage: 199.5+ MB


In [5]:
df_full.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,36805548,39558264,10400,10400,10400,36 months,6.99%,321.08,A,A3,...,0,4,83.3,14.3,0,0,179407,15030,13000,11325
1,38098114,40860827,15000,15000,15000,60 months,12.39%,336.64,C,C1,...,0,4,100.0,0.0,0,0,196500,149140,10000,12000
2,37822187,40585251,9600,9600,9600,36 months,13.66%,326.53,C,C3,...,0,3,100.0,60.0,0,0,52490,38566,21100,24890
3,37662224,40425321,7650,7650,7650,36 months,13.66%,260.20,C,C3,...,0,2,100.0,100.0,0,0,82331,64426,4900,64031
4,37612354,40375473,12800,12800,12800,60 months,17.14%,319.08,D,D4,...,0,0,76.9,100.0,0,0,368700,18007,4400,18000


### Define targets

</br> Initially, try a binary target... <br />1 if total payment received is >= 1.05% of amount funded<br />0 if loan status is 'charged off', 'default', '31-120 days late', or is fully paid, but not >= 1.05% of funded

In [6]:
def final_status(status, funded_amnt, total_pymnt):
    #define list to drop
    drop_status = ['In Grace Period', 'Late (16-30 days)']
    neg_status = ['Late (31-120 days)', 'Charged Off', 'Default']
    
    if status in drop_status:
        return 99
    elif status in neg_status:
        return 0
    elif status == 'Fully Paid':
        if total_pymnt>funded_amnt*1.05:
            return 1
        else:
            return 0
    elif status == 'Current':
        if total_pymnt>funded_amnt*1.05:
            return 1
        else:
            return 99     

In [7]:
df['target'] = df.apply(lambda x: final_status(x['loan_status'], x['funded_amnt_inv'], x['total_pymnt_inv']), axis=1)

In [8]:
df['target'].value_counts()

99    100850
1      93301
0      41478
Name: target, dtype: int64

In [9]:
df.drop(df[df['target']==99].index, inplace=True)

In [10]:
#df.reset_index(inplace=True, drop=True)

In [11]:
#df.set_index(['id'], inplace=True)

In [12]:
df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,target
0,36805548,39558264,10400,10400,10400,36 months,6.99%,321.08,A,A3,...,4,83.3,14.3,0,0,179407,15030,13000,11325,0
1,38098114,40860827,15000,15000,15000,60 months,12.39%,336.64,C,C1,...,4,100.0,0.0,0,0,196500,149140,10000,12000,1
2,37822187,40585251,9600,9600,9600,36 months,13.66%,326.53,C,C3,...,3,100.0,60.0,0,0,52490,38566,21100,24890,0
3,37662224,40425321,7650,7650,7650,36 months,13.66%,260.20,C,C3,...,2,100.0,100.0,0,0,82331,64426,4900,64031,0
5,37842129,40605224,21425,21425,21425,60 months,15.59%,516.36,D,D1,...,2,91.4,100.0,0,0,57073,42315,15000,35573,1


In [13]:
df['id'].describe()

count    1.347790e+05
mean     2.006108e+07
std      8.271381e+06
min      5.716700e+04
25%      1.270694e+07
50%      1.788481e+07
75%      2.704506e+07
max      3.809811e+07
Name: id, dtype: float64

## Clean Data

In [14]:
n_cols = df.shape[1]

In [14]:
drop_cols = [
        'url'
        , 'application_type'
        , 'policy_code'
        , 'id'
        , 'member_id'
    ]

In [15]:
df.drop(drop_cols, axis=1, inplace=True)

In [16]:
df.dropna(how='all', axis=1, inplace=True)

### Choose only columns we will have information for when loan originates

In [17]:
for i in df.columns:
    print (", '{}'".format(i))

, 'loan_amnt'
, 'funded_amnt'
, 'funded_amnt_inv'
, 'term'
, 'int_rate'
, 'installment'
, 'grade'
, 'sub_grade'
, 'emp_title'
, 'emp_length'
, 'home_ownership'
, 'annual_inc'
, 'verification_status'
, 'issue_d'
, 'loan_status'
, 'pymnt_plan'
, 'desc'
, 'purpose'
, 'title'
, 'zip_code'
, 'addr_state'
, 'dti'
, 'delinq_2yrs'
, 'earliest_cr_line'
, 'inq_last_6mths'
, 'mths_since_last_delinq'
, 'mths_since_last_record'
, 'open_acc'
, 'pub_rec'
, 'revol_bal'
, 'revol_util'
, 'total_acc'
, 'initial_list_status'
, 'out_prncp'
, 'out_prncp_inv'
, 'total_pymnt'
, 'total_pymnt_inv'
, 'total_rec_prncp'
, 'total_rec_int'
, 'total_rec_late_fee'
, 'recoveries'
, 'collection_recovery_fee'
, 'last_pymnt_d'
, 'last_pymnt_amnt'
, 'next_pymnt_d'
, 'last_credit_pull_d'
, 'collections_12_mths_ex_med'
, 'mths_since_last_major_derog'
, 'acc_now_delinq'
, 'tot_coll_amt'
, 'tot_cur_bal'
, 'total_rev_hi_lim'
, 'acc_open_past_24mths'
, 'avg_cur_bal'
, 'bc_open_to_buy'
, 'bc_util'
, 'chargeoff_within_12_mths'
, '

In [18]:
drop_cols = [
            'funded_amnt'
            , 'funded_amnt_inv'
            , 'emp_title'
            , 'issue_d'
            , 'loan_status'
            , 'pymnt_plan'
            , 'title'
            , 'zip_code'
            , 'addr_state'
            , 'initial_list_status'
            , 'out_prncp'
            , 'out_prncp_inv'
            , 'total_pymnt'
            , 'total_pymnt_inv'
            , 'total_rec_prncp'
            , 'total_rec_int'
            , 'total_rec_late_fee'
            , 'recoveries'
            , 'collection_recovery_fee'
            , 'last_pymnt_d'
            , 'last_pymnt_amnt'
            , 'next_pymnt_d'
            , 'last_credit_pull_d'
            , 'tot_coll_amt'
            , 'tot_cur_bal'
            ]

In [19]:
df.drop(drop_cols, axis=1, inplace=True)

### Turn into correct data type

In [20]:
df.select_dtypes(exclude=[np.number]).T

,0,1,2,3,5,6,7,8,9,10,...,235616,235618,235619,235620,235621,235622,235623,235625,235626,235628
term,36 months,60 months,36 months,36 months,60 months,36 months,36 months,36 months,36 months,36 months,...,36 months,36 months,36 months,60 months,36 months,36 months,36 months,60 months,36 months,36 months
int_rate,6.99%,12.39%,13.66%,13.66%,15.59%,13.66%,14.31%,11.99%,17.86%,11.44%,...,10.99%,11.99%,15.61%,18.25%,7.90%,18.25%,13.53%,19.97%,7.90%,19.20%
grade,A,C,C,C,D,C,C,B,D,B,...,B,B,C,D,A,D,B,D,A,D
sub_grade,A3,C1,C3,C3,D1,C3,C4,B5,D5,B4,...,B2,B3,C4,D3,A4,D3,B5,D5,A4,D3
emp_length,8 years,10+ years,10+ years,< 1 year,6 years,10+ years,10+ years,< 1 year,10+ years,2 years,...,n/a,8 years,7 years,5 years,3 years,10+ years,4 years,10+ years,3 years,10+ years
home_ownership,MORTGAGE,RENT,RENT,RENT,RENT,MORTGAGE,RENT,MORTGAGE,RENT,RENT,...,MORTGAGE,RENT,RENT,MORTGAGE,RENT,RENT,RENT,MORTGAGE,OWN,MORTGAGE
verification_status,Not Verified,Source Verified,Source Verified,Source Verified,Source Verified,Verified,Source Verified,Source Verified,Source Verified,Not Verified,...,Verified,Source Verified,Verified,Verified,Verified,Source Verified,Source Verified,Verified,Verified,Verified
desc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Borrower added on 12/10/13 > all my loans in...,Borrower added on 12/10/13 > pay off higher ...,NaN,NaN,Borrower added on 12/09/13 > consolidate all...,NaN,NaN,NaN,NaN,Borrower added on 12/04/13 > I will like a l...
purpose,credit_card,debt_consolidation,debt_consolidation,debt_consolidation,credit_card,debt_consolidation,credit_card,home_improvement,house,debt_consolidation,...,credit_card,credit_card,credit_card,medical,debt_consolidation,vacation,credit_card,debt_consolidation,credit_card,other
earliest_cr_line,Sep-1989,Aug-1994,Nov-1992,Aug-2002,Aug-2003,Jan-2001,Oct-1998,Oct-2001,Jan-1999,May-2009,...,May-1990,Aug-1983,Nov-1993,Nov-1994,Jul-1998,Jun-2002,Aug-1999,Jun-1997,Feb-2003,Feb-2000


In [21]:
for i in df.select_dtypes(exclude=[np.number]).columns:
    print (", '{0}'".format(i))

, 'term'
, 'int_rate'
, 'grade'
, 'sub_grade'
, 'emp_length'
, 'home_ownership'
, 'verification_status'
, 'desc'
, 'purpose'
, 'earliest_cr_line'
, 'revol_util'


#### Object to numeric

In [22]:
import re

In [23]:
obj_num_cols = [
        'term'
        , 'int_rate'
        , 'emp_length'
        , 'revol_util'
        ]

In [24]:
df['term'] = df['term'].apply(lambda x: re.sub("[^0-9.,]", '', x))
df['term'] = pd.to_numeric(df['term'])

In [25]:
df['int_rate'] = df['int_rate'].apply(lambda x: re.sub("[^0-9.,]", '', x))
df['int_rate'] = pd.to_numeric(df['int_rate'])

In [26]:
df['emp_length'] = ['0' if x=='< 1 year' else x for x in df['emp_length']]
df['emp_length'] = df['emp_length'].apply(lambda x: re.sub("[^0-9.,]", '', x))
df['emp_length'] = pd.to_numeric(df['emp_length'])#, errors=coerce)

In [27]:
df['revol_util'] = df['revol_util'].apply(lambda x: re.sub("[^0-9.,]", '', str(x)))
df['revol_util'] = pd.to_numeric(df['revol_util'])#, errors=coerce)

#### Object to date

In [28]:
obj_date_cols = [
    #'issue_d'
    'earliest_cr_line'
    ]

In [29]:
df[obj_date_cols] = df[obj_date_cols].apply(lambda x: pd.to_datetime(x))

#### Numeric columns

In [30]:
for i in df.select_dtypes(include=[np.number]).columns:
    print (", '{0}'".format(i))

, 'loan_amnt'
, 'term'
, 'int_rate'
, 'installment'
, 'emp_length'
, 'annual_inc'
, 'dti'
, 'delinq_2yrs'
, 'inq_last_6mths'
, 'mths_since_last_delinq'
, 'mths_since_last_record'
, 'open_acc'
, 'pub_rec'
, 'revol_bal'
, 'revol_util'
, 'total_acc'
, 'collections_12_mths_ex_med'
, 'mths_since_last_major_derog'
, 'acc_now_delinq'
, 'total_rev_hi_lim'
, 'acc_open_past_24mths'
, 'avg_cur_bal'
, 'bc_open_to_buy'
, 'bc_util'
, 'chargeoff_within_12_mths'
, 'delinq_amnt'
, 'mo_sin_old_il_acct'
, 'mo_sin_old_rev_tl_op'
, 'mo_sin_rcnt_rev_tl_op'
, 'mo_sin_rcnt_tl'
, 'mort_acc'
, 'mths_since_recent_bc'
, 'mths_since_recent_bc_dlq'
, 'mths_since_recent_inq'
, 'mths_since_recent_revol_delinq'
, 'num_accts_ever_120_pd'
, 'num_actv_bc_tl'
, 'num_actv_rev_tl'
, 'num_bc_sats'
, 'num_bc_tl'
, 'num_il_tl'
, 'num_op_rev_tl'
, 'num_rev_accts'
, 'num_rev_tl_bal_gt_0'
, 'num_sats'
, 'num_tl_120dpd_2m'
, 'num_tl_30dpd'
, 'num_tl_90g_dpd_24m'
, 'num_tl_op_past_12m'
, 'pct_tl_nvr_dlq'
, 'percent_bc_gt_75'
, 'pub

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 134779 entries, 0 to 235628
Data columns (total 65 columns):
loan_amnt                         134779 non-null int64
term                              134779 non-null int64
int_rate                          134779 non-null float64
installment                       134779 non-null float64
grade                             134779 non-null object
sub_grade                         134779 non-null object
emp_length                        128011 non-null float64
home_ownership                    134779 non-null object
annual_inc                        134779 non-null float64
verification_status               134779 non-null object
desc                              12271 non-null object
purpose                           134779 non-null object
dti                               134779 non-null float64
delinq_2yrs                       134779 non-null int64
earliest_cr_line                  134779 non-null datetime64[ns]
inq_last_6mths           

### Datetime to feature

In [32]:
pd.to_datetime('2014-12-01')

Timestamp('2014-12-01 00:00:00')

In [33]:
df['mo_earliest_credit'] = pd.to_datetime('2014-12-01') - df['earliest_cr_line']

In [34]:
df['mo_earliest_credit'] = df['mo_earliest_credit'].apply(lambda x: x.days/365)

In [35]:
df.drop('earliest_cr_line', axis=1, inplace=True)

### Dummies for object cols

Turn description into a bin for now

In [36]:
df['desc_bin'] = [1 if x else 0 for x in df['desc']]

In [37]:
df.drop(['desc'], axis=1, inplace=True)

In [38]:
obj_cols = df.select_dtypes(include=[object]).columns

In [39]:
df[obj_cols].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 134779 entries, 0 to 235628
Data columns (total 5 columns):
grade                  134779 non-null object
sub_grade              134779 non-null object
home_ownership         134779 non-null object
verification_status    134779 non-null object
purpose                134779 non-null object
dtypes: object(5)
memory usage: 6.2+ MB


In [40]:
df['grade'].value_counts()

C    38552
B    35317
D    24956
A    18148
E    12355
F     4321
G     1130
Name: grade, dtype: int64

In [41]:
df['sub_grade_number'] = df['sub_grade'].str[1]

In [42]:
df['sub_grade_number'].value_counts()

1    27520
4    27078
3    27012
2    26677
5    26492
Name: sub_grade_number, dtype: int64

In [43]:
df['home_ownership'].value_counts()

MORTGAGE    67615
RENT        54160
OWN         13003
ANY             1
Name: home_ownership, dtype: int64

In [44]:
df['verification_status'].value_counts()

Source Verified    54008
Not Verified       40719
Verified           40052
Name: verification_status, dtype: int64

In [45]:
df['purpose'].value_counts()

debt_consolidation    82994
credit_card           29412
home_improvement       7523
other                  6275
major_purchase         2242
medical                1508
small_business         1461
car                    1073
moving                  890
vacation                788
house                   520
renewable_energy         88
wedding                   5
Name: purpose, dtype: int64

Don't use emp_title (too many uniques), pymnt_plan (all the same), title (too many uniques, seems to have been reconsolidated under purpose, don't use address zip code or state for now

In [46]:
for i in obj_cols:
    df = df.merge(pd.get_dummies(df[i]
                                 , prefix='{0}'.format(i))
                , left_index=True
                , right_index=True)

In [47]:
df.drop(obj_cols, axis=1, inplace=True)

In [48]:
df.head()

,loan_amnt,term,int_rate,installment,emp_length,annual_inc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,...,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding
0,10400,36,6.99,321.08,8.0,58000.0,14.92,0,2,42.0,...,0,0,0,0,0,0,0,0,0,0
1,15000,60,12.39,336.64,10.0,78000.0,12.03,0,0,NaN,...,0,0,0,0,0,0,0,0,0,0
2,9600,36,13.66,326.53,10.0,69000.0,25.81,0,0,NaN,...,0,0,0,0,0,0,0,0,0,0
3,7650,36,13.66,260.20,0.0,50000.0,34.81,0,1,NaN,...,0,0,0,0,0,0,0,0,0,0
5,21425,60,15.59,516.36,6.0,63800.0,18.49,0,0,60.0,...,0,0,0,0,0,0,0,0,0,0


In [49]:
def bin_nulls(col, bins):
    bin_num = []
    for func in bins:
        bin_num.append(func(col))
    bin_num = list(set(bin_num))
    bin_num.sort()
    col = pd.cut(col, bin_num, right=True, include_lowest=True)
    dum = pd.get_dummies(col, dummy_na=True)
    return dum

In [50]:
for col in df.columns[df.isnull().any()]:
    df[col] = bin_nulls(df[col], bins=[np.min, partial(np.nanpercentile, q=25), np.nanmedian, partial(np.nanpercentile, q=75), np.max])

In [51]:
np.sum(df.isnull().any())

0

In [52]:
df.to_csv('2014_clean_1.csv')

### Modeling

In [84]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier

In [55]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report

In [56]:
rand = 128

In [57]:
y = df.pop('target')
X = df

In [58]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3)

In [69]:
rfc = RandomForestClassifier(n_estimators=250, max_features='sqrt', n_jobs=-1, class_weight='balanced')

In [85]:
gbc = GradientBoostingClassifier()

In [123]:
gbc_params = dict(n_estimators=[500, 1000]
                 , max_depth=[3]
                 , learning_rate=[.01, .1])

In [124]:
rf_params = dict(n_estimators=[500, 1000]
             , max_features=['sqrt']
             , max_depth=[10, 25, None])

In [125]:
gc = GridSearchCV(gbc, gbc_params, n_jobs=-1, scoring='roc_auc', verbose=5)

In [126]:
gc.fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Done   4 out of  12 | elapsed:  7.1min remaining: 14.2min
[Parallel(n_jobs=-1)]: Done   7 out of  12 | elapsed:  9.5min remaining:  6.8min
[Parallel(n_jobs=-1)]: Done  10 out of  12 | elapsed:  9.8min remaining:  2.0min
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:  9.9min finished


Fitting 3 folds for each of 4 candidates, totalling 12 fits


GridSearchCV(cv=None, error_score='raise',
       estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=100, presort='auto', random_state=None,
              subsample=1.0, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'n_estimators': [500, 1000], 'learning_rate': [0.01, 0.1], 'max_depth': [3]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=5)

In [127]:
gc.cv_results_

{'mean_fit_time': array([ 410.5466667 ,  573.4446667 ,  376.98033333,  546.00333333]),
 'mean_score_time': array([ 2.12366676,  2.03800003,  3.00466665,  2.34500003]),
 'mean_test_score': array([ 0.63936325,  0.64360605,  0.64949711,  0.64723316]),
 'mean_train_score': array([ 0.65162226,  0.66456632,  0.71653528,  0.7569865 ]),
 'param_learning_rate': masked_array(data = [0.01 0.01 0.1 0.1],
              mask = [False False False False],
        fill_value = ?),
 'param_max_depth': masked_array(data = [3 3 3 3],
              mask = [False False False False],
        fill_value = ?),
 'param_n_estimators': masked_array(data = [500 1000 500 1000],
              mask = [False False False False],
        fill_value = ?),
 'params': ({'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 500},
  {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 1000},
  {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 500},
  {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 1000}),
 'r

In [128]:
y_true = y_test
y_pred = gc.predict(X_test)
y_proba = gc.predict_proba(X_test)[:,1]

In [129]:
roc_auc_score(y_true, y_proba)

0.6527994622189055

In [130]:
print (classification_report(y_true, y_pred))

             precision    recall  f1-score   support

          0       0.57      0.16      0.24     12341
          1       0.72      0.95      0.82     28093

avg / total       0.67      0.71      0.64     40434



In [131]:
for i in zip(X_train.columns, gc.best_estimator_.feature_importances_):
    print i

('loan_amnt', 0.026579050442635678)
('term', 0.02122442414705511)
('int_rate', 0.086148359488788656)
('installment', 0.049657937555406421)
('emp_length', 0.0017560104109720976)
('annual_inc', 0.054896019248048065)
('dti', 0.051921414204111428)
('delinq_2yrs', 0.009619282226303618)
('inq_last_6mths', 0.005195840827870943)
('mths_since_last_delinq', 0.0016461527460831515)
('mths_since_last_record', 0.0016763039608906074)
('open_acc', 0.0061184420290275893)
('pub_rec', 0.0031768266478438491)
('revol_bal', 0.047863761289107187)
('revol_util', 0.0022161183557807447)
('total_acc', 0.021492888357869673)
('collections_12_mths_ex_med', 0.0020516556730369993)
('mths_since_last_major_derog', 0.00098163879798915019)
('acc_now_delinq', 0.0002000378411898367)
('total_rev_hi_lim', 0.032310781555116276)
('acc_open_past_24mths', 0.021998680400577501)
('avg_cur_bal', 8.637398820396237e-06)
('bc_open_to_buy', 0.0027073510834937794)
('bc_util', 0.0011150005972646402)
('chargeoff_within_12_mths', 0.0012614

In [132]:
def inspect(X_test, true, pred, proba):
    dfx = X_test.copy()
    dfx['pred'] = pred
    dfx['proba'] = proba
    dfx['true'] = true
    
    return dfx

In [133]:
def grade_inspect(df, grade):
    sliced = df[df['grade_{0}'.format(grade)]==1]
    print classification_report(sliced['true'], sliced['pred'])

In [134]:
x = inspect(X_test, y_test, y_pred, y_proba)

In [135]:
grade_inspect(x, 'A')

             precision    recall  f1-score   support

          0       0.57      0.01      0.02      1389
          1       0.74      1.00      0.85      3994

avg / total       0.70      0.74      0.64      5383



In [136]:
grade_inspect(x, 'B')

             precision    recall  f1-score   support

          0       0.43      0.00      0.01      2341
          1       0.78      1.00      0.88      8368

avg / total       0.70      0.78      0.69     10709



In [137]:
grade_inspect(x, 'C')

             precision    recall  f1-score   support

          0       0.52      0.05      0.08      3460
          1       0.71      0.98      0.82      8211

avg / total       0.65      0.70      0.60     11671



In [138]:
grade_inspect(x, 'D')

             precision    recall  f1-score   support

          0       0.58      0.21      0.30      2754
          1       0.66      0.91      0.76      4638

avg / total       0.63      0.65      0.59      7392



In [139]:
grade_inspect(x, 'E')

             precision    recall  f1-score   support

          0       0.56      0.44      0.49      1647
          1       0.62      0.73      0.67      2068

avg / total       0.59      0.60      0.59      3715



In [140]:
grade_inspect(x, 'F')

             precision    recall  f1-score   support

          0       0.59      0.54      0.56       560
          1       0.64      0.69      0.67       668

avg / total       0.62      0.62      0.62      1228

